# Llama-2-13B-Chat Finetuning on seperate AITA class partitions of Reddit AITA dataset
- Purpose is to learn about the linguistic variations amongst top comments for each AITA class by calculating the perplexity of each finetuned model on test sets for each AITA class.

## Environment set-up

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 107.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
ERROR: pip's dependenc

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/CPSS_24_Paper

Mounted at /content/drive
/content/drive/MyDrive/CPSS_24_Paper


## Load AITA Dataset and Prepare it for Llama-2-Chat Finetuning

In [ ]:
from datasets import load_dataset

dataset = load_dataset("MattBoraske/Reddit-AITA-2018-to-2022")

Generating train split:   0%|          | 0/26407 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2932 [00:00<?, ? examples/s]

In [ ]:
from datasets import Dataset

def filter_aita_class(example):
    return example['top_comment_1_AITA_class_by_keyword'] == 'NTA'

train_dataset = dataset['train'].filter(filter_aita_class)

Filter:   0%|          | 0/26407 [00:00<?, ? examples/s]

In [ ]:
columns_to_drop = ['submission_score', 'submission_url', 'submission_date', 'top_comment_2', 'top_comment_3', 'top_comment_4', 'top_comment_5', 'top_comment_6', 'top_comment_7', 'top_comment_8', 'top_comment_9', 'top_comment_10', 'top_comment_1_AITA_class_by_keyword', 'top_comment_2_AITA_class_by_keyword', 'top_comment_3_AITA_class_by_keyword', 'top_comment_4_AITA_class_by_keyword', 'top_comment_5_AITA_class_by_keyword', 'top_comment_6_AITA_class_by_keyword', 'top_comment_7_AITA_class_by_keyword', 'top_comment_8_AITA_class_by_keyword', 'top_comment_9_AITA_class_by_keyword', 'top_comment_10_AITA_class_by_keyword']
train_dataset = train_dataset.remove_columns(columns_to_drop)

In [ ]:
def create_input_text(example):
    example['input_text'] = example['submission_title'] + " " + example['submission_text']
    return example

# Apply the function to create the new column
train_dataset = train_dataset.map(create_input_text)

Map:   0%|          | 0/20834 [00:00<?, ? examples/s]

In [ ]:
train_dataset = train_dataset.remove_columns(['submission_title', 'submission_text'])
train_dataset = train_dataset.rename_column('top_comment_1', 'reference_text')

# Swap the positions of the columns
columns = train_dataset.column_names
new_columns_order = ['input_text', 'reference_text']  # Change to the desired order
train_dataset = Dataset.from_pandas(train_dataset.select(range(train_dataset.num_rows)).to_pandas()[new_columns_order])

In [ ]:
train_dataset

Dataset({
    features: ['input_text', 'reference_text'],
    num_rows: 20834
})

In [ ]:
def get_llama2_training_instruction(sample):
    llama2_instruction = f"<s>[INST] {sample['input_text']} [/INST] {sample['reference_text']} </s>"
    return {"llama2_instruction": llama2_instruction}

train_dataset = train_dataset.map(get_llama2_training_instruction)

columns_to_drop = ['input_text', 'reference_text']
train_dataset = train_dataset.remove_columns(columns_to_drop)
train_dataset = train_dataset.rename_column('llama2_instruction', 'input_text')

Map:   0%|          | 0/20834 [00:00<?, ? examples/s]

## Finetune Llama-2-Chat

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

KeyboardInterrupt: 

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-13b-chat-hf"

# Fine-tuned model name
new_model = "llama-2-13b-chat-reddit-AITA-NTA"

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.05

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model logs and checkpoints will be stored
output_dir = f"{new_model}/logs_and_checkpoints"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Log every X updates steps
logging_steps = 10

# Log first step
logging_first_step = True

# Logging strategy
logging_strategy = "steps"

# Save checkpoint every X updates steps
save_steps = 100

# Save strategy
save_strategy = "steps"

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 4096

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = '[PAD]'
tokenizer.padding_side = "right" # fix overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    save_strategy=save_strategy,
    logging_steps=logging_steps,
    logging_first_step=logging_first_step,
    logging_strategy=logging_strategy,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="input_text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

In [ ]:
# Train model
trainer.train()

# Save peft model locally for future merging with base model
trainer.model.save_pretrained(new_model)